In [58]:
import covidcast as covid
import pandas as pd

### Global variables: re-fetch data or read from local csv files

In [83]:
fetch_from_covidcast = False
regenerate_state_fips = False
regenerate_county_fips = False
remerge_covid_and_fips_data = False
regenerate_usda_poverty_data = False
regenerate_usda_unemployment_median_hhi_data = False
regenerate_usda_population_data = False
regenerate_usda_education_data = False

### Fetch raw covid data and save to csv

In [60]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
if fetch_from_covidcast:
    confirmed_cumulative_cases_prop = covid.signal(data_source="usa-facts", signal="confirmed_cumulative_prop", geo_type="county")
    confirmed_daily_incidence_cases_prop = covid.signal(data_source="usa-facts", signal="confirmed_incidence_prop", geo_type="county")
    cumulative_deaths_prop = covid.signal(data_source="usa-facts", signal="deaths_cumulative_prop", geo_type="county")
    daily_incidence_deaths_prop = covid.signal(data_source="usa-facts", signal="deaths_incidence_prop", geo_type="county")
else:
    confirmed_cumulative_cases_prop = pd.read_csv("data/covid_usafacts/raw/confirmed_cumulative_cases_prop.csv", index_col=0)
    confirmed_daily_incidence_cases_prop = pd.read_csv("data/covid_usafacts/raw/confirmed_daily_incidence_cases_prop.csv", index_col=0)
    cumulative_deaths_prop = pd.read_csv("data/covid_usafacts/raw/cumulative_deaths_prop.csv", index_col=0)
    daily_incidence_deaths_prop = pd.read_csv("data/covid_usafacts/raw/daily_incidence_deaths_prop.csv", index_col=0)
    

### Retreive FIPS codes for each US county

In [62]:
if regenerate_state_fips:
    state_fips = pd.read_excel("data/fips/raw/state_fips_2019.xlsx", skiprows=range(5))
    state_fips = state_fips[["State (FIPS)", "Name"]]
    state_fips = state_fips.rename(columns={"State (FIPS)": "State FIPS"})
    state_fips.to_csv("data/fips/clean/state_fips_2019.csv")
else:
    state_fips = pd.read_csv("data/fips/clean/state_fips_2019.csv", index_col=0)

In [63]:
if regenerate_county_fips:
    fips = pd.read_excel('data/fips/raw/area_fips_2019.xlsx', skiprows=range(4))
    county_fips = fips[fips["Summary Level"]==50]
    county_fips = county_fips.rename(columns={"Area Name (including legal/statistical area description)": "Area Name",
                                              "State Code (FIPS)": "State FIPS",
                                              "County Code (FIPS)": "County FIPS"})
    county_fips["Area Name"] = county_fips["Area Name"].str.title()
    county_fips["FIPS"] = county_fips["State FIPS"].apply(str).str.zfill(2) + county_fips["County FIPS"].apply(str).str.zfill(3)
    county_fips["FIPS"] = county_fips["FIPS"].astype('int64')
    county_fips = county_fips[["State FIPS", "County FIPS", "FIPS", "Area Name"]]
    county_fips = county_fips.merge(state_fips, how='left', left_on="State FIPS", right_on="State FIPS")
    county_fips = county_fips.rename(columns={"Name": "State Name"})
    county_fips = county_fips[["State FIPS", "County FIPS", "FIPS", "State Name", "Area Name"]]
    county_fips.to_csv("data/fips/clean/county_fips_2019.csv")
else:
    county_fips = pd.read_csv("data/fips/clean/county_fips_2019.csv", index_col=0)    

### Merge covid datasets with fips data

In [64]:
def merge_covid_and_county_fips_dfs(covid_df, county_fips_df):
    df = covid_df.merge(county_fips_df, how='left', left_on='geo_value', right_on='FIPS')
    return df[["FIPS", "time_value", "issue", "value", "State Name", "Area Name"]]

In [84]:
if remerge_covid_and_fips_data:
    confirmed_cumulative_cases_prop_fips = merge_covid_and_county_fips_dfs(confirmed_cumulative_cases_prop, county_fips)
    confirmed_daily_incidence_cases_prop_fips = merge_covid_and_county_fips_dfs(confirmed_daily_incidence_cases_prop, county_fips)
    cumulative_deaths_prop_fips = merge_covid_and_county_fips_dfs(cumulative_deaths_prop, county_fips)
    daily_incidence_deaths_prop_fips = merge_covid_and_county_fips_dfs(daily_incidence_deaths_prop, county_fips)
    
    confirmed_cumulative_cases_prop_fips.to_csv("data/covid_usafacts/clean/confirmed_cumulative_cases_props_fips.csv")
    confirmed_daily_incidence_cases_prop_fips.to_csv("data/covid_usafacts/clean/confirmed_daily_incidence_cases_prop_fips.csv")
    cumulative_deaths_prop_fips.to_csv("data/covid_usafacts/clean/cumulative_deaths_prop_fips.csv")
    daily_incidence_deaths_prop_fips.to_csv("data/covid_usafacts/clean/daily_incidence_deaths_prop_fips.csv")

else:
    confirmed_cumulative_cases_prop_fips = pd.read_csv("data/covid_usafacts/clean/confirmed_cumulative_cases_props_fips.csv", index_col=0)
    confirmed_daily_incidence_cases_prop_fips = pd.read_csv("data/covid_usafacts/clean/confirmed_daily_incidence_cases_prop_fips.csv", index_col=0)
    cumulative_deaths_prop_fips = pd.read_csv("data/covid_usafacts/clean/cumulative_deaths_prop_fips.csv", index_col=0)
    daily_incidence_deaths_prop_fips = pd.read_csv("data/covid_usafacts/clean/daily_incidence_deaths_prop_fips.csv", index_col=0)

In [117]:
daily_incidence_deaths_prop_fips

,FIPS,time_value,issue,value,State Name,Area Name
0,1001,2020-01-25,2020-10-17,0.0,Alabama,Autauga County
1,1003,2020-01-25,2020-10-17,0.0,Alabama,Baldwin County
2,1005,2020-01-25,2020-10-17,0.0,Alabama,Barbour County
3,1007,2020-01-25,2020-10-17,0.0,Alabama,Bibb County
4,1009,2020-01-25,2020-10-17,0.0,Alabama,Blount County
...,...,...,...,...,...,...
892323,56037,2020-11-10,2020-11-12,0.0,Wyoming,Sweetwater County
892324,56039,2020-11-10,2020-11-12,0.0,Wyoming,Teton County
892325,56041,2020-11-10,2020-11-12,0.0,Wyoming,Uinta County
892326,56043,2020-11-10,2020-11-12,0.0,Wyoming,Washakie County


In [137]:
x = daily_incidence_deaths_prop_fips
x["time_value"] = pd.to_datetime(x["time_value"])
x = x[x["FIPS"]//1000==36]
x.groupby(['FIPS', 'Area Name'])['value'].agg(Min='min', Max='max', Average='mean', Median='median').reset_index()

,FIPS,Area Name,Min,Max,Average,Median
0,36001,Albany County,0.000000,6.873842,0.152137,0.000000
1,36003,Allegany County,0.000000,8.678484,0.221546,0.000000
2,36005,Bronx County,-0.098716,21.252187,1.245224,0.169228
3,36007,Broome County,0.000000,3.674772,0.216272,0.000000
4,36009,Cattaraugus County,0.000000,2.627534,0.097145,0.000000
...,...,...,...,...,...,...
57,36115,Washington County,0.000000,9.803281,0.080543,0.000000
58,36117,Wayne County,0.000000,2.224249,0.031327,0.000000
59,36119,Westchester County,0.000000,13.023175,0.538629,0.000000
60,36121,Wyoming County,0.000000,7.526531,0.044170,0.000000


### Fetch and clean USDA County-level data

#### Poverty Estimates (2018)

In [66]:
if regenerate_usda_poverty_data:
    poverty = pd.read_excel("data/usda_county_datasets/raw/PovertyEstimates.xls", skiprows=range(4))
    poverty = poverty[["FIPStxt", "Stabr", "Area_name", "PCTPOVALL_2018", "PCTPOV017_2018"]]
    poverty = poverty.rename(columns={"FIPStxt": "FIPS", 
                            "Stabr": "State Abrv", 
                            "Area_name": "Area Name", 
                            "PCTPOVALL_2018": "% Total Population in Poverty (2018)",
                            "PCTPOV017_2018": "% Children Ages 0-17 in Poverty (2018)"})
    poverty.to_csv("data/usda_county_datasets/clean/poverty_2018.csv")
else:
    poverty = pd.read_csv("data/usda_county_datasets/clean/poverty_2018.csv", index_col=0)

#### Unemployment and Median Household Income Estimates (2018)

In [67]:
if regenerate_usda_unemployment_median_hhi_data:
    unemployment_median_hhi = pd.read_excel("data/usda_county_datasets/raw/Unemployment.xls", skiprows=range(7))
    unemployment_median_hhi = unemployment_median_hhi[["FIPStxt", "Stabr", "area_name", "Unemployment_rate_2018", "Median_Household_Income_2018", "Med_HH_Income_Percent_of_State_Total_2018"]]
    unemployment_median_hhi = unemployment_median_hhi.rename(columns={"FIPStxt": "FIPS",
                                            "Stabr": "State Abrv",
                                            "area_name": "Area Name",
                                            "Unemployment_rate_2018": "% Unemployment (2018)",
                                            "Median_Household_Income_2018": "Median Household Income (2018)",
                                            "Med_HH_Income_Percent_of_State_Total_2018": "Median Household Income % of State Total (2018)"})
    unemployment_median_hhi.to_csv("data/usda_county_datasets/clean/unemployment_median_hhi_2018.csv")
else:
    unemployment_median_hhi = pd.read_csv("data/usda_county_datasets/clean/unemployment_median_hhi_2018.csv", index_col=0)
    

#### Population Estimates (2018)

In [68]:
if regenerate_usda_population_data:
    population = pd.read_excel("data/usda_county_datasets/raw/PopulationEstimates.xls", skiprows=range(2))
    population = population[["FIPStxt", "State", "Area_Name", "POP_ESTIMATE_2018"]]
    population = population.rename(columns={"FIPStxt": "FIPS",
                               "State": "State Abrv",
                               "Area_Name": "Area Name",
                               "POP_ESTIMATE_2018": "Population Estimate (2018)"})
    population.to_csv("data/usda_county_datasets/clean/population_2018.csv")
else:
    population = pd.read_csv("data/usda_county_datasets/clean/population_2018.csv", index_col=0)

#### Education Estimates (2014-2018)

In [69]:
if regenerate_usda_education_data:
    education = pd.read_excel("data/usda_county_datasets/raw/Education.xls", skiprows=range(4))
    education = education[["FIPS Code", "State", "Area name", 
                           "Percent of adults with less than a high school diploma, 2014-18",
                           "Percent of adults with a high school diploma only, 2014-18",
                           "Percent of adults completing some college or associate's degree, 2014-18",
                           "Percent of adults with a bachelor's degree or higher, 2014-18"]]
    education = education.rename(columns={"FIPS Code": "FIPS",
                              "State": "State Abrv",
                              "Area name": "Area Name",
                              "Percent of adults with less than a high school diploma, 2014-18": "% Adults Incomplete High School (2018)",
                              "Percent of adults with a high school diploma only, 2014-18": "% Adults Complete High School (2018)",
                              "Percent of adults completing some college or associate's degree, 2014-18": "% Adults Some College (2018)",
                              "Percent of adults with a bachelor's degree or higher, 2014-18": "% Adults Complete College (2018)"})
    education.to_csv("data/usda_county_datasets/clean/education_2018.csv")
else:
    education = pd.read_csv("data/usda_county_datasets/clean/education_2018.csv", index_col=0)

#### Get most recent cumulative covid data for given state

In [ ]:
# def get_cumulative_covid_data_for_state(state, confirmed_cumulative_cases_county, cumulative_deaths_county):
#     state_cumulative_deaths = cumulative_deaths_county[cumulative_deaths_county["State Name"]==state]
#     state_cumulative_deaths = state_cumulative_deaths[~state_cumulative_deaths.duplicated("geo_value", keep="last")]
#     state_cumulative_deaths = state_cumulative_deaths.rename(columns={"geo_value": "FIPS", "value": "Cumulative Deaths"})
#     state_cumulative_deaths = state_cumulative_deaths[["FIPS", "Area Name", "time_value", "issue", "Cumulative Deaths"]]
    
#     state_confirmed_cumulative_cases = confirmed_cumulative_cases_county[confirmed_cumulative_cases_county["State Name"]==state]
#     state_confirmed_cumulative_cases = state_confirmed_cumulative_cases[~state_confirmed_cumulative_cases.duplicated("geo_value", keep="last")]
#     state_confirmed_cumulative_cases = state_confirmed_cumulative_cases.rename(columns={"geo_value": "FIPS", "value": "Confirmed Cumulative Cases"})
#     state_confirmed_cumulative_cases = state_confirmed_cumulative_cases[["FIPS", "Area Name", "time_value", "issue", "Confirmed Cumulative Cases"]]
    
#     state_cumulative = state_cumulative_deaths.merge(state_confirmed_cumulative_cases, how='inner', on="FIPS", suffixes=('', '_'))
#     state_cumulative = state_cumulative.drop(columns=[col for col in state_cumulative.columns if col.endswith("_")])
#     return state_cumulative

In [ ]:
# ny_covid_cumulative = get_cumulative_covid_data_for_state("New York", confirmed_cumulative_cases_county, cumulative_deaths_county)
# ny_covid_cumulative.to_csv("data/covid_counties/ny_cumulative.csv")

In [ ]:
# pa_covid_cumulative = get_cumulative_covid_data_for_state("Pennsylvania", confirmed_cumulative_cases_county, cumulative_deaths_county)
# pa_covid_cumulative.to_csv("data/covid_counties/pa_cumulative.csv")

#### Get daily incidence covid data for given state

In [ ]:
# def get_daily_covid_incidence_for_state(state, incidence_col_name, daily_incidence_county_df):
#     state_daily_incidence = daily_incidence_county_df[daily_incidence_county_df["State Name"]==state]
#     state_daily_incidence = state_daily_incidence.rename(columns={"geo_value": "FIPS", "value": incidence_col_name})
#     state_daily_incidence = state_daily_incidence[["FIPS", "Area Name", "time_value", "issue", incidence_col_name]]
#     state_daily_incidence = state_daily_incidence.reset_index(drop=True)
#     return state_daily_incidence

In [ ]:
# ny_daily_cases = get_daily_covid_incidence_for_state("New York", "Daily Confirmed Cases", confirmed_daily_incidence_cases_county)
# ny_daily_cases.to_csv("data/covid_counties/ny_daily_cases.csv")

# ny_daily_deaths = get_daily_covid_incidence_for_state("New York", "Daily Deaths", daily_incidence_deaths_county)
# ny_daily_deaths.to_csv("data/covid_counties/ny_daily_deaths.csv")

In [ ]:
# pa_daily_cases = get_daily_covid_incidence_for_state("Pennsylvania", "Daily Confirmed Cases", confirmed_daily_incidence_cases_county)
# pa_daily_cases.to_csv("data/covid_counties/pa_daily_cases.csv")

# pa_daily_deaths = get_daily_covid_incidence_for_state("Pennsylvania", "Daily Deaths", daily_incidence_deaths_county)
# pa_daily_deaths.to_csv("data/covid_counties/pa_daily_deaths.csv")